In [1]:
import json
import pickle
import re
import os
from collections import defaultdict
from konlpy.tag import Mecab

In [3]:
mecab = Mecab()

In [26]:
output_dir = '/opt/ml/output/test-lstm-top12-concat'

In [27]:
with open(os.path.join(output_dir, 'nbest_predictions.json'), 'r') as f:
    nbest = json.load(f)

In [28]:
blacklist = ['최소', '대략', '한때', '간혹', '최소한', '필수']

def valid_answer(text):
    # This function only tests for answers with Korean
    if not re.search('[가-힣]', text):
        return True
    elif text.strip() in blacklist:
        return False
        
    part_of_speech = [x[1][0] for x in mecab.pos(text)]

    # TODO: M tag 수식언만 나오는 경우? 
    if len(part_of_speech) == 1 and 'M' in part_of_speech:
        return False
    return True
    # return 'N' in part_of_speech

In [29]:
candidates = defaultdict(list)

predictions = {}
pattern = re.compile('[.\",\{\}\[\]\(\)\<\>]')

for mrc_id in nbest.keys():
    nbest_preds = [(p['text'], p['probability']) for p in nbest[mrc_id]]
    sorted_scores = sorted(nbest_preds, key=lambda x:x[1], reverse=True)  # already done?

    selected = 0
    # 정답이 next highest score 로 넘어가는 경우:
    # 1. 괄호, 부호 등을 제외하면 빈 string 일 때 
    # 2. 정답의 길이가 30자를 넘어갈 때
    # 3. 정답에 체언을 포함하지 않을 때
    pred = sorted_scores[selected][0]
    while (re.sub(pattern, '', pred) == '' or len(pred) > 30 or not valid_answer(pred)) and selected < len(sorted_scores)-1:
        selected += 1
        pred = sorted_scores[selected][0]
    predictions[mrc_id] = pred

In [30]:
with open(os.path.join(output_dir, 'filtered_predictions.json'), 'w') as f:
    json.dump(predictions, f, ensure_ascii=False, indent = 4)